In [13]:
import dgl
import numpy as np
import torch

In [14]:
edge_file_path = "../cleaned_data/graph_data/edge_ID.txt"
node_type_file_path = "../cleaned_data/graph_data/node_type_ID.txt"

In [ ]:
import pandas as pd

In [21]:
node_type_index = {}  # ID: type
# 两类节点，四类边
user_node_set = []
poi_node_set = []

with open(node_type_file_path) as f:
    lines = f.readlines()
    for line in lines:
        line_content = line.strip().split("\t")
        node = line_content[0]
        node_type = line_content[1]
        node_type_index[node] = node_type

        if node_type == "P":
            poi_node_set.append(node)
        elif node_type == "U":
            user_node_set.append(node)

In [22]:
user_node_set

['OhECKhQEexFypOMY6kypRw',
 '4hBhtCSgoxkrFgHa4YAD-w',
 'UsBxLh14sUpO8SdeqIiGOA',
 'C_2mNjl-doRVvsL03_T57Q',
 'RiQWbgwR-asiU_mA7s8gpg',
 'RgtbLaiU22zqaCk20HgbiQ',
 'jvURGq9nyZkTF1KuSaZC_g',
 'FNf6kM1wfeBeB2UW6pif7g',
 'Uk3X2AypU8AqvcYEVf7s6Q',
 'gasLVm0KRwrVhPGRcqATjw',
 'VOWlOVEBg9LoHykSBue7EA',
 'yv-x6ga91bklbhrHFPoRYA',
 'w90KNnzS6Ao3k-njRlCEeA',
 'DAXuBz8W8I0P6Fqden3kjA',
 'jytULgxk9gDRuZQy0AyusQ',
 'WGuhb1GYI_13pX_e92A7zg',
 '1hfMnNQ7tl49puuAz-UJFw',
 'lAlUjkkyzQAAWAo4BNs2Hw',
 'vIr2vYWApBXiQv5qHFn6gQ',
 'zg5PKSWdi6ZVZpOzkiuhFA',
 'SVO8tY3ryARNFb_bX4VVBg',
 'SrfDRvGKI8FQq9LCr0dQuQ',
 'o6UJMpHcpLJEvmKLrxLS3w',
 'RMmcPwhkkivzm80a2rs9LQ',
 '2uGGrCtfoAju9tdj5RHEpw',
 'qurv1eJZswxUZaZqN53Meg',
 'gZ7mbHLlQsLd6S_i5U69tQ',
 'cZ16H_Fc3s2-2AYrs-LEJg',
 'UaCNJLCNlsz0y6gXvGPBhQ',
 'SVywIrtEdtIeGmq_OGl2iw',
 'l2IWzTJtrIlP_W9fHit1cA',
 'h5-NsPUzUPd2cS4DEEIr4g',
 'O_upLtNFSEim4QFeMd2sSQ',
 'a_iFcnIHwsb9ww81kRYirA',
 'AePwMGMiiy12E2dSTV7vdA',
 'Bys8OhL5Q0uMXymBk5zRzw',
 '-pOPETmDQKXdp_0dtn1kXg',
 

In [23]:
user_node_set_map = {
    node_id: i for i, node_id in enumerate(user_node_set)
}  # 对于每一类节点，dgl中都是从0 开始索引的
poi_node_set_map = {node_id: i for i, node_id in enumerate(poi_node_set)}

In [24]:
"""
在DGL的图里，所有的边都是有方向的， 即边 (u,v)  表示它是从节点 u  指向节点 v  的。
所以最后是有四种边
"""
count = 0
user_user_edge = []
user_user_edge_weight = []

poi_poi_edge = []
poi_poi_edge_weight = []

user_poi_edge = []  #
user_poi_edge_weight = []

poi_user_edge = []  #
poi_user_edge_weight = []

with open(edge_file_path) as f:
    lines = f.readlines()
    for line in lines:
        line_content = line.strip().split(",")
        source_node = line_content[0]
        target_node = line_content[1]
        edge_weight = eval(line_content[2])
        if (  # POI to POI
            node_type_index[source_node] == "P" and node_type_index[target_node] == "P"
        ):  # POI之间的边
            poi_poi_edge.append(
                (poi_node_set_map[source_node], poi_node_set_map[target_node])
            )
            # poi_poi_edge.append(
            #     (poi_node_set_map[target_node], poi_node_set_map[source_node])
            # )  # 此行把 POI边变成双向的

            poi_poi_edge_weight.append(edge_weight)
            # poi_poi_edge_weight.append(edge_weight)

        elif (  # User to User
            node_type_index[source_node] == "U" and node_type_index[target_node] == "U"
        ):  # user之间的边
            user_user_edge.append(
                (user_node_set_map[source_node], user_node_set_map[target_node])
            )
            user_user_edge.append(
                (user_node_set_map[target_node], user_node_set_map[source_node])
            )  # 此行把 User边变成双向的

            user_user_edge_weight.append(edge_weight)
            user_user_edge_weight.append(
                edge_weight
            )  # 边重复了两次，权值自然要重复两次
        else:
            if node_type_index[source_node] == "U":
                user_poi_edge.append(
                    (user_node_set_map[source_node], poi_node_set_map[target_node])
                )
                # edge_weight = edge_weight * (dic[(source_node, target_node)] - 1) * 0.25

                user_poi_edge_weight.append(edge_weight)

                poi_user_edge.append(
                    (poi_node_set_map[target_node], user_node_set_map[source_node])
                )
                poi_user_edge_weight.append(edge_weight)
            else:  # source_node P
                poi_user_edge.append(
                    (poi_node_set_map[source_node], user_node_set_map[target_node])
                )
                # edge_weight = edge_weight * (dic[(source_node, target_node)] - 1) * 0.25
                poi_user_edge_weight.append(edge_weight)

                user_poi_edge.append(
                    (user_node_set_map[target_node], poi_node_set_map[source_node])
                )
                user_poi_edge_weight.append(edge_weight)

In [25]:
user_user_edge_array = np.array(user_user_edge)
user_user_edge_weight_array = np.array(user_user_edge_weight)  #

poi_poi_edge_array = np.array(poi_poi_edge)
poi_poi_edge_weight_array = np.array(poi_poi_edge_weight)  #

user_poi_edge_array = np.array(user_poi_edge)  #
user_poi_edge_weight_array = np.array(user_poi_edge_weight)  #

poi_user_edge_array = np.array(poi_user_edge)  #
poi_user_edge_weight_array = np.array(poi_user_edge_weight)  #

In [26]:
graph_data = {
    ("user", "user_user", "user"): (
        torch.tensor(user_user_edge_array[:, 0]),
        torch.tensor(user_user_edge_array[:, 1]),
    ),
    ("user", "user_poi", "poi"): (
        torch.tensor(user_poi_edge_array[:, 0]),
        torch.tensor(user_poi_edge_array[:, 1]),
    ),
    ("poi", "poi_user", "user"): (
        torch.tensor(poi_user_edge_array[:, 0]),
        torch.tensor(poi_user_edge_array[:, 1]),
    ),
    ("poi", "poi_poi", "poi"): (
        torch.tensor(poi_poi_edge_array[:, 0]),
        torch.tensor(poi_poi_edge_array[:, 1]),
    ),
}
g = dgl.heterograph(graph_data)  # 构建图

In [27]:
print(g.ntypes)
print(g.metagraph().edges())
print("user_num_nodes", g.num_nodes("user"))
print("poi_num_nodes", g.num_nodes("poi"))
print(g.etypes)
print(g.canonical_etypes)

['poi', 'user']
[('poi', 'poi'), ('poi', 'user'), ('user', 'poi'), ('user', 'user')]
user_num_nodes 6021
poi_num_nodes 1226
['poi_poi', 'poi_user', 'user_poi', 'user_user']
[('poi', 'poi_poi', 'poi'), ('poi', 'poi_user', 'user'), ('user', 'user_poi', 'poi'), ('user', 'user_user', 'user')]


In [28]:
# 添加边特征
g.edges["user_user"].data["w"] = torch.tensor(
    user_user_edge_weight_array, dtype=torch.float32
)
g.edges["user_poi"].data["w"] = torch.tensor(
    user_poi_edge_weight_array, dtype=torch.float32
)
g.edges["poi_user"].data["w"] = torch.tensor(
    poi_user_edge_weight_array, dtype=torch.float32
)
g.edges["poi_poi"].data["w"] = torch.tensor(
    poi_poi_edge_weight_array, dtype=torch.float32
)

In [ ]:
g

Graph(num_nodes={'poi': 1226, 'user': 6021},
      num_edges={('poi', 'poi_poi', 'poi'): 40194, ('poi', 'poi_user', 'user'): 53035, ('user', 'user_poi', 'poi'): 53035, ('user', 'user_user', 'user'): 77392},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])

In [ ]:
poi_fe = np.load("../cleaned_data/graph_data/poi_fe.npy")
user_fe = np.load("../cleaned_data/graph_data/user_fe.npy")

In [ ]:
g.nodes["user"].data["fe"] = torch.tensor(user_fe, dtype=torch.float32)
g.nodes["poi"].data["fe"] = torch.tensor(poi_fe, dtype=torch.float32)

In [18]:
torch.save(g, "a-g.bin")

In [19]:
m = torch.load("a-g.bin")

/tmp/ipykernel_3272716/742473614.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m = torch.load("a-g.bin")


In [20]:
m

Graph(num_nodes={'poi': 1226, 'user': 6021},
      num_edges={('poi', 'poi_poi', 'poi'): 40194, ('poi', 'poi_user', 'user'): 53035, ('user', 'user_poi', 'poi'): 53035, ('user', 'user_user', 'user'): 77392},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])